In [186]:
import re
import os
import bcolz
%matplotlib inline
import numpy as np
import pandas as pd
from pprint import pprint
import _pickle as cPickle
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.utils.data_utils import get_file
from keras.layers import SimpleRNN, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D
from keras.layers import Dense, Dropout, Activation, Flatten
from numpy.random import random, permutation, randn, normal, uniform, choice
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional

In [187]:
# path = get_file(fname = 'allPyPrograms_no_comments.txt', origin = 'file:///homw/wtemp/585/allPyPrograms_no_comments.txt')
path = os.getcwd() + '/allPyPrograms_no_comments.txt'
text_all = open(path).read()
text = text_all[:1000000]
print(len(text))

1000000


In [188]:
from pprint import pprint
pprint(text[:300])

('def sort_blocks():\n'
 "    with open('README.md', 'r') as read_me_file:\n"
 '        read_me = read_me_file.read()\n'
 "    table_of_contents = ''.join(read_me.split('- - -')[0])\n"
 "    blocks = ''.join(read_me.split('- - -')[1]).split('\\n# ')\n"
 '    for i in range(len(blocks)):\n'
 '        if i == 0:\n'
 '            blocks[i]')


In [189]:
chars = sorted(list(set(text)))
print(len(chars)+1)

247


In [190]:
chars.insert(0, '\0')

In [191]:
char_to_index = {v:i for i,v in enumerate(chars)}
index_to_char = {i:v for i,v in enumerate(chars)}

In [192]:
total_index = [char_to_index[char] for char in text]

In [193]:
total_index[:10]

[70, 71, 72, 2, 85, 81, 84, 86, 65, 68]

In [194]:
''.join(index_to_char[i] for i in total_index[:25])

'def sort_blocks():\n    wi'

In [195]:
pred_num = 3
xin = [[total_index[j+i] for j in range(0, len(total_index)-1-pred_num, pred_num)] for i in range(pred_num)]
y = [total_index[i+pred_num] for i in range(0, len(total_index)-1-pred_num, pred_num)]

In [196]:
X = [np.stack(xin[i][:-2]) for i in range(pred_num)]
Y = np.stack(y[:-2])

In [197]:
pprint(X)

[array([70,  2, 84, ...,  2,  2, 75]),
 array([71, 85, 86, ...,  2,  9, 71]),
 array([72, 81, 65, ...,  2, 88, 89])]


In [198]:
Y[:8]

array([ 2, 84, 68, 69, 10,  1,  2, 75])

In [199]:
X[0].shape, Y.shape

((333330,), (333330,))

In [200]:
hidden_layers = 256
vocab_size = 4118
n_fac = 42

In [201]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, activation='relu'),
        Dense(vocab_size, activation='softmax')
    ])

In [202]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 3, 42)             172956    
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_7 (Dense)              (None, 4118)              1058326   
Total params: 1,307,826
Trainable params: 1,307,826
Non-trainable params: 0
_________________________________________________________________


In [203]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [204]:
model.fit(np.stack(X, 1), Y, batch_size=64, epochs=5)

Epoch 1/5
333330/333330 [==============================] - 193s 578us/step - loss: 2.2740
Epoch 2/5
333330/333330 [==============================] - 195s 585us/step - loss: 1.7429
Epoch 3/5
333330/333330 [==============================] - 192s 577us/step - loss: 1.6107
Epoch 4/5
333330/333330 [==============================] - 191s 574us/step - loss: 1.5451
Epoch 5/5
333330/333330 [==============================] - 194s 583us/step - loss: 1.5040


In [211]:
model.save_weights('simpleRNN_3pred.h5')

In [212]:
model.load_weights('simpleRNN_3pred.h5')

In [213]:
model.save_weights('simpleRNN_7pred.h5')

In [214]:
model.load_weights('simpleRNN_7pred.h5')

In [ ]:
def predict_next_char(inp):
    index = [char_to_index[i] for i in inp]
    arr = np.expand_dims(np.array(index), axis=0)
    prediction = model.predict(arr)
    return index_to_char[np.argmax(prediction)]

In [229]:
predict_next_char(' de')

'f'

In [230]:
predict_next_char('t34')

'1'

In [236]:
prog = "while True:"
for i in range(1000):
    prog = prog + predict_next_char(prog[-3:]).replace('\n', ' ')
prog

"while True:  # self._ext': '\\u226', ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''), ''

In [219]:
ys = [[total_index[j+i] for j in range(1, len(total_index)-pred_num, pred_num)] for i in range(pred_num)]

In [220]:
Y_return = [np.stack(ys[i][:-2]) for i in range(pred_num)]

In [221]:
X

[array([70,  2, 84, ...,  2,  2, 75]),
 array([71, 85, 86, ...,  2,  9, 71]),
 array([72, 81, 65, ...,  2, 88, 89])]

In [222]:
Y_return

[array([71, 85, 86, ...,  2,  9, 71]),
 array([72, 81, 65, ...,  2, 88, 89]),
 array([ 2, 84, 68, ...,  2, 75, 65])]

In [223]:
vocab_size = 97
n_fac = 42
hidden_layers = 256

In [224]:
return_model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, return_sequences=True, activation='relu'),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])

In [225]:
return_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 3, 42)             4074      
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 3, 256)            76544     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 3, 97)             24929     
Total params: 105,547
Trainable params: 105,547
Non-trainable params: 0
_________________________________________________________________


In [226]:
return_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [227]:
X_model = np.stack(X, 1)
Y_model = np.expand_dims(np.stack(Y_return, 1), axis=-1)

In [228]:
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

Epoch 1/5
   320/333330 [..............................] - ETA: 14:52 - loss: 4.5533  

InvalidArgumentError: indices[11,1] = 142 is not in [0, 97)
	 [[Node: embedding_8/embedding_lookup = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@embedding_8/embeddings"], validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_8/embeddings/read, embedding_8/Cast)]]

Caused by op 'embedding_8/embedding_lookup', defined at:
  File "/home/wtemp/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wtemp/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wtemp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/wtemp/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/wtemp/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home

In [ ]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

In [ ]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

In [ ]:
return_model.save_weights('return_sequences_25.h5')

In [166]:
def predict_every_char(inp):
    l = []
    p = 0
    while p<len(inp):
        pre_inp = inp[p:p+pred_num]
        if len(pre_inp) < pred_num:
            pre_inp = pre_inp + ' '*(pred_num - len(pre_inp))
            l.append(pre_inp)
        else:
            l.append(pre_inp) 
        p+=pred_num

#     index = [char_to_index[i] for i in inp]
#     arr = np.expand_dims(index, axis=0)
#     prediction = return_model.predict(arr)
#     return ''.join([index_to_char[np.argmax(i)] for i in prediction[0]])
    
    final = []
    for half in l:
        index = [char_to_index[i] for i in half]
        arr = np.expand_dims(index, axis=0)
        prediction = return_model.predict(arr)
        final.append(''.join([index_to_char[np.argmax(i)] for i in prediction[0]]))
    
    return ''.join(final)

In [185]:
predict_every_char('qor item in items:')

'uag n rptn rnhme.\n'